In [ ]:
from time import time, gmtime, strftime
import numpy as np
import os
import random

import scipy.io.wavfile as wav
from python_speech_features import mfcc

import tensorflow as tf
import keras


from cs224s_accent_conversion.tf.utils.general_utils import get_minibatches, batch_multiply_by_matrix
from cs224s_accent_conversion.tf.utils.fast_dtw import get_dtw_series 
from cs224s_accent_conversion.tf.utils.pad_sequence import pad_sequence

from keras.models import Sequential
from keras.layers import Dropout, Dense, BatchNormalization, Activation
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam, SGD, RMSprop

In [ ]:
def preprocess_data(num_mfcc_coeffs, max_num_frames):
    
    inputs = [] 
    labels = [] 
    input_masks = []
    label_masks = []
    
    SOURCE_DIR = 'ACCENT_CONVERSION_DATASET/us-english-male-bdl/wav/'
    TARGET_DIR = 'ACCENT_CONVERSION_DATASET/indian-english-male-ksp/wav/'
    for source_fname, target_fname in zip(os.listdir(SOURCE_DIR), os.listdir(TARGET_DIR)):
        (source_sample_rate, source_wav_data) = wav.read(SOURCE_DIR + source_fname) 
        (target_sample_rate, target_wav_data) = wav.read(TARGET_DIR + target_fname)

        source_mfcc_features = np.array(mfcc(source_wav_data, samplerate=source_sample_rate, numcep=num_mfcc_coeffs))
        target_mfcc_features = np.array(mfcc(target_wav_data, samplerate=target_sample_rate, numcep=num_mfcc_coeffs))

        # Aligns the MFCC features matrices using FastDTW.
        source_mfcc_features, target_mfcc_features = get_dtw_series(source_mfcc_features, target_mfcc_features)

        # Pads the MFCC feature matrices (rows) to length config.max_num_frames
        source_padded_frames, source_mask = pad_sequence(source_mfcc_features, max_num_frames)
        target_padded_frames, target_mask = pad_sequence(target_mfcc_features, max_num_frames)

        inputs.append(source_padded_frames) 
        input_masks.append(source_mask)
        labels.append(target_padded_frames) 
        label_masks.append(target_mask) 

    randomized_indices = list(range(0, len(inputs))) 
    random.shuffle(randomized_indices)
    inputs = [inputs[i] for i in randomized_indices]
    input_masks = [input_masks[i] for i in randomized_indices]
    labels = [labels[i] for i in randomized_indices]
    label_masks = [label_masks[i] for i in randomized_indices] 

    return inputs, labels

In [ ]:
batch_size = 32 
n_epochs = 50
lr = 1e-3
max_num_frames = 1220 
num_mfcc_coeffs = 25
sample_rate = 16000.0
num_features = max_num_frames * num_mfcc_coeffs

state_size_1 = 25
state_size_2 = 50
state_size_3 = 50
state_size_4 = 25

In [ ]:
inputs, labels = preprocess_data(num_mfcc_coeffs, max_num_frames)

In [ ]:
model = Sequential()

model.add(Dense(state_size_1, input_shape = (25,))) 
model.add(Activation('relu'))

model.add(Dense(state_size_2))
model.add(Activation('relu'))

model.add(Dense(state_size_3))
model.add(Activation('relu'))

model.add(Dense(state_size_4))
model.add(Activation('relu'))
          
model.add(Dense(25))

In [ ]:
optimizer = Adam(lr = lr)
model.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics = ['accuracy'])
model.summary()

In [ ]:
np_array = np.array(inputs)
m = np_array.shape[2]
print(np_array.shape)
inputs = np.reshape(inputs, [-1, 25])
print(inputs.shape)

np.array1 = np.array(labels)
print(np_array.shape)
labels = np.reshape(inputs, [-1, 25])
print(labels.shape)

In [ ]:
history = model.fit(x = inputs, y = labels, batch_size = batch_size*1220, epochs = n_epochs, verbose = 1, validation_split = 0.05)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['acc'])
plt.title('Model Acc')
plt.ylabel('Acc')
plt.xlabel('Epoch')
plt.show()

In [ ]:
# Plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['acc'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_acc'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
from librosa import display
def plot(x, p):
    plt.figure(figsize = (10, 4))
    display.specshow(x, x_axis = 'time')
    plt.colorbar()
    plt.title(str(i+1) + 'MFCC' + p)

In [ ]:
for i in range(30):
    a = predictions[i*1220:(i+1)*1220, :]
    a_y = test_labels[i*1220:(i+1)*1220, :]
    plot(a_y.T, "(target)")
    plot(a.T, "(predicted)")

In [3]:
class myclass ():
    def func(self,config):
        print(config.var)
class Config():
    var = 0
